# Inference

In [18]:
import os
import pandas as pd
from PIL import Image
from ultralytics import YOLO

# Load a pretrained YOLO model
# model = YOLO("/home/oem/ayeong/yolo11x_pseudo/yolov11x_ay_concat4/weights/best.pt") # 025로 pseudo한거
# model = YOLO("/home/oem/ayeong/yolo11_kfold/train4/weights/best.pt")
# model = YOLO("/home/oem/ayeong/yolo11_kfold/train42/weights/best.pt")
# model = YOLO("/home/oem/ayeong/yolo11_kfold/train422/weights/best.pt")
# model = YOLO("/home/oem/ayeong/yolo11_kfold/train4222/weights/best.pt")
model = YOLO( "/home/oem/ayeong/yolo11_kfold/train42222/weights/best.pt")



# Define path to the image file
source = "/home/oem/ayeong/dataset/test"
output_csv = './yolo11_fold4.csv'

# Initialize variables for submission
prediction_strings = []
file_names = []
score_threshold = 0.05  # Set your desired score threshold

In [19]:
# Iterate through each test image
for img_file in os.listdir(source):
    img_path = os.path.join(source, img_file)
    
    # Open image
    img = Image.open(img_path)

    # Run inference on the resized image
    results = model(img)  # Run inference
    
    prediction_string = ''
    
    # Iterate through detected boxes
    for result in results:  # results is a list of Result objects
        for box, score, label in zip(result.boxes.xyxy, result.boxes.conf, result.boxes.cls):
            if score > score_threshold:
                # Convert YOLO box format [x1, y1, x2, y2] to PASCAL format
                x1 = box[0].item()  # x1
                y1 = box[1].item()  # y1
                x2 = box[2].item()  # x2
                y2 = box[3].item()  # y2
                
                # Append to prediction string in desired format
                # prediction_string += f"{int(label.item())} {score:.6f} {x_min:.4f} {y_min:.4f} {x_max:.4f} {y_max:.4f} "
                prediction_string += f"{int(label.item())} {score:.6f} {x1:.4f} {y1:.4f} {x2:.4f} {y2:.4f} "
    
    prediction_strings.append(prediction_string.strip())
    file_names.append(img_file)  # Use the file name directly


0: 640x640 2 General trashs, 1 Paper, 2 Plastics, 3 Plastic bags, 8.2ms
Speed: 1.2ms preprocess, 8.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 General trashs, 1 Paper, 1 Plastic, 8.2ms
Speed: 1.1ms preprocess, 8.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 General trash, 1 Paper, 8.2ms
Speed: 1.1ms preprocess, 8.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 General trash, 5 Papers, 1 Plastic bag, 8.2ms
Speed: 1.1ms preprocess, 8.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 7 General trashs, 1 Paper pack, 8.2ms
Speed: 1.1ms preprocess, 8.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 General trash, 8.2ms
Speed: 1.1ms preprocess, 8.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 7 Papers, 1 Paper pack, 1 Plastic, 4 Plastic bags, 8.2ms
Speed: 1.1ms preprocess, 8.2ms infere

In [20]:
# Create submission DataFrame
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission['image_id'] = 'test/'+ submission['image_id']
submission = submission.sort_values('image_id')

# Save to CSV
submission.to_csv(output_csv, index=False)
print(submission.head())

                                       PredictionString       image_id
2032  7 0.978025 604.6456 518.1860 957.0257 1024.000...  test/0000.jpg
3079  6 0.973175 765.6777 665.1020 898.7697 988.1094...  test/0001.jpg
1465  1 0.973197 302.0190 319.0675 1020.9160 746.022...  test/0002.jpg
4190  9 0.929467 143.3874 261.0173 911.4969 819.3303...  test/0003.jpg
4168  0 0.649579 196.4767 251.8317 871.1372 778.2319...  test/0004.jpg
